In [2]:
import pandas as pd
from sqlalchemy import create_engine
from Postgres_Login import username, password


In [3]:
rds_connection_string = f"{username}:{password}@localhost:5432/streaming_services"
engine = create_engine(f'postgresql://{rds_connection_string}')


In [4]:
filePath = "Resources/HuluRaw_AP.csv"
data = pd.read_csv(filePath, encoding='utf-8')


In [5]:
dataCleaned = data[['show/name','show/genre','show/seasons_count',
                    'show/show_rollups/showtime/highest_rating']]
dataCleaned.head()

,show/name,show/genre,show/seasons_count,show/show_rollups/showtime/highest_rating
0,Family Guy,Animation and Cartoons,15,TV-MA
1,South Park,Comedy,20,TV-MA
2,American Dad!,Comedy,9,TV-14
3,Law & Order: Special Victims Unit,Drama,18,TV-14
4,Naruto Shippuden,Anime,9,TV-14


In [8]:
dataCleaned = dataCleaned.rename(columns={'show/name':'Title of Show',
                                          'show/genre':'Genres',
                                          'show/seasons_count':'No. of Seasons',
                                          'show/show_rollups/showtime/highest_rating':'TV Rating'})
dataCleaned['Hulu'] = 'Yes'
dataCleaned['TV Rating'].fillna("Not Rated", inplace = True)
dataCleaned

,Title of Show,Genres,No. of Seasons,TV Rating,Hulu
0,Family Guy,Animation and Cartoons,15,TV-MA,Yes
1,South Park,Comedy,20,TV-MA,Yes
2,American Dad!,Comedy,9,TV-14,Yes
3,Law & Order: Special Victims Unit,Drama,18,TV-14,Yes
4,Naruto Shippuden,Anime,9,TV-14,Yes
...,...,...,...,...,...
995,East Los High,Drama,4,TV-14,Yes
996,Sesame Street,Kids,3,TV-Y,Yes
997,Casual,Comedy,3,TV-MA,Yes
998,Chicago P.D.,Drama,1,TV-14,Yes


In [9]:
dataCleaned.to_sql(name='hulu', con=engine, if_exists='append', index=False)
pd.read_sql_query('select * from hulu', con=engine).head()



,Title of Show,Genres,No. of Seasons,TV Rating,Hulu
0,Family Guy,Animation and Cartoons,15,TV-MA,Yes
1,South Park,Comedy,20,TV-MA,Yes
2,American Dad!,Comedy,9,TV-14,Yes
3,Law & Order: Special Victims Unit,Drama,18,TV-14,Yes
4,Naruto Shippuden,Anime,9,TV-14,Yes
